# ChatBot
In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

---

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
# openai.api_type = os.environ.get("OPENAI_API_TYPE")
# openai.api_base = os.environ.get("OPENAI_API_BASE")
# openai.api_version = os.environ.get("OPENAI_API_VERSION")
openai.api_key = os.environ.get("OPENAI_API_KEY")

## Helper Function
<img src="../../images/role.png" alt="Role" style="width: 50%; height: auto;"/>

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## The Chat Format

In [4]:
messages = [
    {"role": "system", "content": "You are an assistant that speaks like Shakespeare."},
    {"role": "user", "content": "tell me a joke"},
    {"role": "assistant", "content": "Why did the chicken cross the road"},
    {"role": "user", "content": "I don't know"},
]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To forsoothly get to the other side, my dear.


In [6]:
messages = [
    {"role": "system", "content": "You are friendly chatbot."},
    {"role": "user", "content": "Hi, my name is Alice"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Alice! It's a pleasure to meet you. How can I assist you today?


In [7]:
messages = [
    {"role": "system", "content": "You are friendly chatbot."},
    {"role": "user", "content": "Yes,  can you remind me, What is my name?"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as an AI language model, I don't have access to your personal information, including your name. Could you please tell me how you would like me to address you?


In [11]:
messages = [
    {"role": "system", "content": "You are friendly chatbot."},
    {"role": "user", "content": "Hi, my name is Alice"},
    {
        "role": "assistant",
        "content": "Hi Alice! It's nice to meet you. \
Is there anything I can help you with today?",
    },
    {"role": "user", "content": "Yes, you can remind me, What is my name?"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Alice.


## OrderBot
We can automate the collection of user prompts and assistant responses to build a OrderBot. The OrderBot will take orders at a pizza restaurant.

In [12]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ""
    context.append({"role": "user", "content": f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({"role": "assistant", "content": f"{response}"})
    panels.append(pn.Row("User:", pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row(
            "Assistant:",
            pn.pane.Markdown(
                response, width=600, style={"background-color": "#F6F6F6"}
            ),
        )
    )

    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI

pn.extension()

panels = []  # collect display

context = [
    {
        "role": "system",
        "content": """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
""",
    }
]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here…")
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages = context.copy()
messages.append(
    {
        "role": "system",
        "content": "create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price ",
    },
)
# The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)